In [ ]:
import carla, time, pygame, math, random, cv2
import numpy as np

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.load_world("Town04")
spectator = world.get_spectator()

# Functions
def move_spectator_to(transform, distance=1.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def draw_on_screen(world, transform, content='O', color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), width=800, height=600):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

map = world.get_map()
map

Constants used in the following scripts.

In [ ]:
EGO_VEHICLE_INITIAL_LOCATION = carla.Location(x=280, y=-207.5, z=0.1) # z=0.1 used to make the car not clip with the ground
EGO_VEHICLE_INITIAL_ROTATION = carla.Rotation(yaw=180)

TRAFFIC_VECHICLE_RIGHT_LANE_POSITION = (285, -227.5)

Spawn the ego vehicle in a parking slot, with the spectator facing the back of the vehicle.

In [ ]:
for actor in world.get_actors().filter("vehicle.*"):
    actor.destroy()

ego_vehicle = spawn_vehicle()
time.sleep(1)
print(ego_vehicle.get_transform().location)
print(world.get_map().get_spawn_points()[0])

# Set the vehicle position
ego_vehicle_transform = ego_vehicle.get_transform()
ego_vehicle_transform.location = EGO_VEHICLE_INITIAL_LOCATION
ego_vehicle_transform.rotation = EGO_VEHICLE_INITIAL_ROTATION
ego_vehicle.set_transform(ego_vehicle_transform)

# Set the spectator camera
spectator_transform = ego_vehicle.get_transform()
spectator_transform.location = ego_vehicle_transform.location + carla.Location(x=2)
spectator_transform.rotation = carla.Rotation(yaw=180)
move_spectator_to(spectator_transform)

Spawn some traffic vehicles

In [ ]:
def spawn_vehicle_right_lane(
        x=285, y=-227.5,
        #time_to_live=4,
        spawn_delay=0,
        start_delay=1):
    time.sleep(spawn_delay)
    traffic_vehicle = spawn_vehicle()

    traffic_vehicle_transform = traffic_vehicle.get_transform()
    traffic_vehicle_transform.location = carla.Location(x=x, y=y)
    traffic_vehicle_transform.rotation = carla.Rotation(yaw=91.5) # with yaw=90 it's not aligned with the parking slots
    traffic_vehicle.set_transform(traffic_vehicle_transform)

    time.sleep(start_delay)
    traffic_vehicle.apply_control(carla.VehicleControl(throttle=random.uniform(0.7, 1)))

    # time.sleep(time_to_live)
    # traffic_vehicle.destroy()
    return traffic_vehicle

def spawn_vehicle_left_lane(
        x=286, y=-192,
        #time_to_live=4,
        spawn_delay=0,
        start_delay=1):
    time.sleep(spawn_delay)
    traffic_vehicle = spawn_vehicle()

    traffic_vehicle_transform = traffic_vehicle.get_transform()
    traffic_vehicle_transform.location = carla.Location(x=x, y=y, z=0.5)
    traffic_vehicle_transform.rotation = carla.Rotation(yaw=271.5)
    traffic_vehicle.set_transform(traffic_vehicle_transform)

    time.sleep(start_delay)
    traffic_vehicle.apply_control(carla.VehicleControl(throttle=random.uniform(0.7, 1)))

    # time.sleep(time_to_live)
    # traffic_vehicle.destroy()
    return traffic_vehicle

def spawn_two_vehicles(time_to_live=4):
    vehicle1_destroy = True
    vehicle2_destroy = True
    try:
        while(True):
            #time.sleep(random.uniform(0, 5))
            if(vehicle1_destroy):
                vehicle1 = spawn_vehicle_right_lane()
                vehicle1_destroy = False
            #time.sleep(random.uniform(0, 5))
            if(vehicle2_destroy):
                vehicle2 = spawn_vehicle_left_lane()
                vehicle2_destroy = False
            
            #time.sleep(time_to_live)
            if(vehicle1.get_transform().location.y > -192):
                vehicle1_destroy = True

            if(vehicle2.get_transform().location.y < -227.5):
                vehicle2_destroy = True

            if(vehicle1_destroy):
                vehicle1.destroy()
            if(vehicle2_destroy):
                vehicle2.destroy()
    finally:
       vehicle1.destroy()
       vehicle2.destroy()

spawn_two_vehicles()

In [ ]:
for v in world.get_actors().filter('vehicle.*'):
    v.destroy()

Spawn a pedestrian

In [55]:
def spawn_pedestrians_right_lane(
        x=283,
        y=-213,
        spawn_delay=0,
        start_delay=1):
    time.sleep(spawn_delay)

    blueprint_library = world.get_blueprint_library()
    walker_bp = random.choice(blueprint_library.filter('walker.pedestrian.*'))
    spawn_points = world.get_map().get_spawn_points()
    spawn_point = random.choice(spawn_points)
    pedestrian = world.spawn_actor(walker_bp, spawn_point)

    pedestrian_transform = pedestrian.get_transform()
    pedestrian_transform.location = carla.Location(x=x, y=y, z=0.5)
    pedestrian_transform.rotation = carla.Rotation(yaw=-91.5)
    pedestrian.set_transform(pedestrian_transform)

    time.sleep(start_delay)
    pedestrian.apply_control(carla.WalkerControl(direction=carla.Vector3D(0, 1, 0), speed=random.uniform(0.9, 1)))

    return pedestrian

def spawn_pedestrian():
    pedestrian_spawn = True
    try:
        while(True):
            if(pedestrian_spawn):
                try:
                    pedestrian = spawn_pedestrians_right_lane()
                    pedestrian_spawn = False
                except RuntimeError as e:
                    print(f"RuntimeError: {e}")
            try:    
                if(pedestrian.get_location().y > -203.5):
                    pedestrian.destroy()
                    pedestrian_spawn = True
            except RuntimeError as e:
                print(f"RuntimeError: {e}")
    finally:
        pedestrian.destroy()

spawn_pedestrian()

# while True:
#     move_spectator_to(pedestrian.get_transform())
#     world.tick()
#     time.sleep(0.01)


KeyboardInterrupt: 

In [54]:
for p in world.get_actors().filter('walker.pedestrian.*'):
    p.destroy()

In [ ]:
ego_vehicle = spawn_vehicle()

time.sleep(1)

#target_transform = target_vehicle.get_transform()
#target_transform.location += carla.Location(x=100)
#target_vehicle.set_transform(target_transform)

ego_vehicle = spawn_vehicle()
ego_vehicle_transform = ego_vehicle.get_transform()
ego_vehicle_transform.location = carla.Location(x=283, y=-206)
ego_vehicle.set_transform(ego_vehicle_transform)

camera = spawn_camera(attach_to=ego_vehicle)

video_output = np.zeros((600, 800, 4), dtype=np.uint8)
def camera_callback(image):
    global video_output
    video_output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

camera.listen(lambda image: camera_callback(image))

# target_vehicle.set_autopilot(True)

ego_vehicle.apply_control(carla.VehicleControl(throttle = 1))

# target_velocity = carla.Vector3D(x=20) # Set the target velocity to 20 m/s
# ego_vehicle.set_target_velocity(target_velocity)

cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)

running = True

try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        cv2.imshow('RGB Camera', video_output)
finally:
    cv2.destroyAllWindows()
    camera.destroy()
    ego_vehicle.destroy()
    ego_vehicle.destroy()